In [1]:
import os
import sys
import torch
import random
import torchmetrics

sys.path.append(os.path.abspath(os.path.join(r'C:\Users\incognito\OneDrive\Desktop\data_mining')))

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from models.cnn import CNN
from torchvision import transforms
from models.resnet import ResNet
from torch.utils.data import DataLoader, TensorDataset
from utils.data_utils import count_unique_colum_and_vlaues

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
train_features = np.load("../data/processed/balance_16k/model_input/train_features.npy")
train_labels = np.load("../data/processed/balance_16k/model_input/train_labels.npy")

test_features = np.load("../data/processed/balance_16k/model_input/test_features.npy")
test_labels = np.load("../data/processed/balance_16k/model_input/test_labels.npy")


In [4]:
train_features = torch.from_numpy(train_features).float()
train_features = train_features.to(device)
train_labels = torch.from_numpy(train_labels)
train_labels = train_labels.to(device)

test_features = torch.from_numpy(test_features).float()
test_features = test_features.to(device)
test_labels = torch.from_numpy(test_labels)
test_labels = test_labels.to(device)

In [5]:
print(f"Train features: {train_features.shape}, Train label: {train_labels.shape} ")
print(f"Test features: {test_features.shape}, Test label: {test_labels.shape} ")
train_min, train_max = train_features.min(), train_features.max()
print((train_min, train_max))

# inputs = 2 * (inputs - x_min) / (x_max - x_min) -1

Train features: torch.Size([14672, 3, 50, 50]), Train label: torch.Size([14672]) 
Test features: torch.Size([1631, 3, 50, 50]), Test label: torch.Size([1631]) 
(tensor(0., device='cuda:0'), tensor(13373320., device='cuda:0'))


In [6]:
unique_value, count = count_unique_colum_and_vlaues(train_labels.to(device='cpu'))
for x in range(len(count)):
    print(unique_value[x], count[x])

0 2713
1 2703
2 2712
3 2676
4 1185
5 2683


In [7]:
batch_size = 32
dataset = TensorDataset(train_features, train_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_features, test_labels)
test_dataloader = DataLoader(test_dataset)

In [8]:
# Hyperparameters

# our defined resnet model's performance based best learning rate.
# initial_lr = 1e-4
# base_lr = 1e-4

initial_lr = 1e-4
base_lr = 1e-4
warmup_steps = 10 
epochs = 20

model = CNN()
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [9]:
def train_model(
    model=model,
    dataloader=dataloader,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=epochs,
    warmup_steps=warmup_steps,
    initial_lr=initial_lr,
    base_lr=base_lr,
):
    model = model.to(device=device)

    model.train()
    for epoch in range(epochs):
        # if epoch < warmup_steps:
        #     lr = initial_lr + (base_lr - initial_lr) * (epoch / warmup_steps)
        #     for param_group in optimizer.param_groups:
        #         param_group["lr"] = lr
        # else:
        #     scheduler.step()
        #     lr = scheduler.get_last_lr()[0]

        batch_loss = 0.0
        for batch_inputs, batch_targets in dataloader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()

            batch_loss += loss.item()

        avg_loss = batch_loss / len(dataloader)
        print(
            f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {avg_loss:.4f}, Learning Rate: {initial_lr:.6f}"
        )

In [10]:
train_model()

Epoch 1/20, Cross Entropy Loss: 1.3857, Learning Rate: 0.000100
Epoch 2/20, Cross Entropy Loss: 1.2740, Learning Rate: 0.000100
Epoch 3/20, Cross Entropy Loss: 1.2146, Learning Rate: 0.000100
Epoch 4/20, Cross Entropy Loss: 1.1670, Learning Rate: 0.000100
Epoch 5/20, Cross Entropy Loss: 1.1250, Learning Rate: 0.000100
Epoch 6/20, Cross Entropy Loss: 1.0930, Learning Rate: 0.000100
Epoch 7/20, Cross Entropy Loss: 1.0601, Learning Rate: 0.000100
Epoch 8/20, Cross Entropy Loss: 1.0325, Learning Rate: 0.000100
Epoch 9/20, Cross Entropy Loss: 1.0116, Learning Rate: 0.000100
Epoch 10/20, Cross Entropy Loss: 0.9888, Learning Rate: 0.000100
Epoch 11/20, Cross Entropy Loss: 0.9638, Learning Rate: 0.000100
Epoch 12/20, Cross Entropy Loss: 0.9446, Learning Rate: 0.000100
Epoch 13/20, Cross Entropy Loss: 0.9259, Learning Rate: 0.000100
Epoch 14/20, Cross Entropy Loss: 0.9065, Learning Rate: 0.000100
Epoch 15/20, Cross Entropy Loss: 0.8914, Learning Rate: 0.000100
Epoch 16/20, Cross Entropy Loss: 0

In [11]:
def calculate_accuracy(model, dataloader, num_classes):
    model.eval()
    total_correct = 0
    total_samples = 0
    class_correct = torch.zeros(num_classes)
    class_samples = torch.zeros(num_classes)

    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            
            outputs = model(inputs)
            _, predictions = torch.max(
                outputs, 1
            )

            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

            for i in range(len(labels)):
                label = labels[i]
                if predictions[i] == label:
                    class_correct[label] += 1
                class_samples[label] += 1

    total_accuracy = total_correct / total_samples * 100
    class_accuracy = class_correct / class_samples * 100

    return total_accuracy, class_accuracy


num_classes = 6
total_accuracy, class_accuracy = calculate_accuracy(
    model, num_classes=6, dataloader=test_dataloader
)

print(f"Total Accuracy: {total_accuracy:.2f}%")
for i in range(num_classes):
    print(f"Accuracy of class {i}: {class_accuracy[i]:.2f}%")

Total Accuracy: 65.67%
Accuracy of class 0: 63.41%
Accuracy of class 1: 73.40%
Accuracy of class 2: 71.53%
Accuracy of class 3: 62.35%
Accuracy of class 4: 33.05%
Accuracy of class 5: 70.66%


In [12]:
# model.eval()

# test_dataset = TensorDataset(test_features, test_labels)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device)

# with torch.no_grad():
#     for batch_test_inputs, batch_test_targets in test_dataloader:
#         outputs = model(batch_test_inputs)
#         predictions = torch.argmax(outputs, dim=1)
#         accuracy.update(predictions, batch_test_targets)

# final_accuracy = accuracy.compute().item()
# print(f"Accuracy on the test data: {final_accuracy * 100:.2f}%")

# accuracy.reset()


# def lr_finder(model, criterion, optimizer, train_loader, init_value=1e-10, final_value=10, beta=0.98):
#     num = len(train_loader) - 1
#     mult = (final_value / init_value) ** (1/num)
#     lr = init_value
#     optimizer.param_groups[0]['lr'] = lr
#     avg_loss, best_loss = 0.0, 0.0
#     losses, log_lrs = [], []

#     for i, (inputs, labels) in enumerate(train_loader):
#         # Move data to the correct device
#         inputs, labels = inputs.to(device), labels.to(device)

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Compute the smoothed loss
#         avg_loss = beta * avg_loss + (1 - beta) * loss.item()
#         smoothed_loss = avg_loss / (1 - beta ** (i + 1))

#         # Stop if the loss is exploding
#         if i > 1 and smoothed_loss > 4 * best_loss:
#             break

#         # Record the best loss
#         if smoothed_loss < best_loss or i == 0:
#             best_loss = smoothed_loss

#         # Backward pass and update the weights
#         loss.backward()
#         optimizer.step()

#         # Store the values
#         losses.append(smoothed_loss)
#         log_lrs.append(lr)

#         # Update the learning rate
#         lr *= mult
#         optimizer.param_groups[0]['lr'] = lr

#     # Plot the learning rate vs loss graph
#     plt.plot(log_lrs, losses)
#     plt.xscale('log')
#     plt.xlabel("Learning Rate")
#     plt.ylabel("Loss")
#     plt.show()

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-8)

# lr_finder(model, criterion, optimizer, dataloader)